# Task 5:

In [1]:
import pandas as pd
import joblib
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler,PowerTransformer
from sklearn.decomposition import PCA

Load Data

In [2]:
df_raw = pd.read_csv('test.csv')
df = df_raw.copy()
df.drop(['id', 'player_id'], axis=1, inplace=True)
df

,login_count_1,login_count_2,login_count_3,login_count_4,login_lat_1,login_lon_1,login_lat_2,login_lon_2,login_lat_3,login_lon_3,...,guild_changes_1,guild_changes_2,guild_changes_3,guild_changes_4,account_age_days,behavior_stability,password_changed,vpn_usage,suspicious_login_time,mass_item_sale
0,31.222232,41.790417,28.048736,NaN,-17.074035,127.285635,-17.554811,126.865553,-18.679380,128.290929,...,1.0,2.0,2.0,1.0,580.020715,0.830282,0.0,1.0,0.0,0.0
1,61.358999,49.209284,NaN,NaN,48.430879,25.555743,48.708993,26.733951,48.706072,26.597056,...,0.0,-1.0,-1.0,1.0,1306.627890,0.700770,0.0,NaN,0.0,0.0
2,24.763478,37.723228,26.879628,32.503285,8.252684,40.788695,8.346856,40.688039,8.630887,40.910033,...,0.0,1.0,-1.0,0.0,835.638378,0.797991,0.0,0.0,0.0,0.0
3,82.100747,74.862388,73.520927,83.495309,-28.728293,178.990465,-29.104994,178.810800,-28.198793,180.045695,...,2.0,2.0,1.0,3.0,NaN,0.736845,0.0,NaN,0.0,0.0
4,50.901216,70.636572,55.606357,68.179388,-46.833704,167.886402,-47.372141,166.767058,-47.828834,167.372518,...,NaN,-1.0,0.0,0.0,379.053416,0.818901,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25884,58.476294,61.813452,61.044533,69.714151,10.926213,173.299402,11.431859,172.835916,10.898828,172.888824,...,2.0,3.0,1.0,2.0,1448.835875,0.741764,0.0,1.0,0.0,0.0
25885,59.206258,80.713010,66.253998,71.432253,-20.608607,-13.291896,-20.767979,-12.663475,-20.211497,-11.923994,...,2.0,3.0,2.0,3.0,1132.604300,0.877150,0.0,1.0,0.0,0.0
25886,94.317618,74.329146,107.617203,85.256921,-59.318059,-55.518951,-58.671733,-56.658019,-58.829435,-55.885138,...,2.0,3.0,1.0,2.0,822.011236,0.702062,0.0,0.0,NaN,0.0
25887,78.325845,66.700837,58.570940,NaN,-26.220106,-131.999085,-25.175901,-131.024958,-25.462090,-132.055088,...,1.0,1.0,2.0,1.0,1162.971652,0.749088,0.0,0.0,0.0,0.0


In [3]:
cat_col = [
    'device_count_1',
    'device_count_2',
    'device_count_3',
    'device_count_4',
    'refund_count_1',
    'refund_count_2',
    'refund_count_3',
    'refund_count_4',
    'guild_changes_1',
    'guild_changes_2',
    'guild_changes_3',
    'guild_changes_4',
    'password_changed',
    'vpn_usage',
    'suspicious_login_time',
    'mass_item_sale',
]

In [4]:
num_col = df.columns.difference(cat_col)

Missing Value Imputation IterativeImputer

In [5]:
for col in cat_col:
  df[col] = df[col].fillna(df[col].mode()[0])

In [6]:
for col in num_col:
  df[col] = df[col].fillna(df[col].median())

Scaling data

In [8]:
scaler = StandardScaler()
df_2 = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
df_2

,login_count_1,login_count_2,login_count_3,login_count_4,login_lat_1,login_lon_1,login_lat_2,login_lon_2,login_lat_3,login_lon_3,...,guild_changes_1,guild_changes_2,guild_changes_3,guild_changes_4,account_age_days,behavior_stability,password_changed,vpn_usage,suspicious_login_time,mass_item_sale
0,-1.172321,-0.694309,-1.272356,-0.027409,-0.507350,1.257268,-0.521818,1.252540,-0.558975,1.263964,...,-0.010478,0.878938,0.873266,-0.574365,-0.710267,0.134409,-0.247295,2.346574,-0.224101,-0.164982
1,0.093969,-0.397719,-0.055492,-0.027409,1.438112,0.250109,1.446424,0.263228,1.437405,0.259705,...,-1.254813,-1.785353,-1.784413,-0.574365,1.241164,-1.325894,-0.247295,-0.426153,-0.224101,-0.164982
2,-1.443706,-0.856907,-1.320300,-1.144926,0.244840,0.400920,0.247543,0.401096,0.250127,0.401051,...,-1.254813,-0.009159,-1.784413,-1.629031,-0.023761,-0.229689,-0.247295,-0.426153,-0.224101,-0.164982
3,0.965498,0.627835,0.592446,1.052319,-0.853476,1.769162,-0.864894,1.765764,-0.841000,1.775059,...,1.233857,0.878938,-0.012627,1.534967,0.009849,-0.919134,-0.247295,-0.426153,-0.224101,-0.164982
4,-0.345447,0.458897,-0.142225,0.392356,-1.391197,1.659229,-1.407485,1.646771,-1.422565,1.649907,...,-0.010478,-1.785353,-0.898520,-1.629031,-1.250001,0.006074,-0.247295,-0.426153,-0.224101,-0.164982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25884,-0.027157,0.106168,0.080793,0.458489,0.324243,1.712819,0.339177,1.706732,0.317317,1.704383,...,1.233857,1.767034,-0.012627,0.480301,1.623089,-0.863672,-0.247295,2.346574,-0.224101,-0.164982
25885,0.003515,0.861731,0.294432,0.532522,-0.612325,-0.134495,-0.617259,-0.126023,-0.604366,-0.120702,...,1.233857,1.767034,0.873266,1.534967,0.773793,0.662863,-0.247295,2.346574,-0.224101,-0.164982
25886,1.478828,0.606518,1.990725,1.128227,-1.761976,-0.552557,-1.743119,-0.560693,-1.748472,-0.554832,...,1.233857,1.767034,-0.012627,0.480301,-0.060359,-1.311329,-0.247295,-0.426153,-0.224101,-0.164982
25887,0.806884,0.301555,-0.020648,-0.027409,-0.778984,-1.309735,-0.748188,-1.295447,-0.759922,-1.307035,...,-0.010478,-0.009159,0.873266,-0.574365,0.855350,-0.781098,-0.247295,-0.426153,-0.224101,-0.164982


In [9]:
pt = PowerTransformer(method='yeo-johnson')

for col in num_col:
    skew_val = df_2[col].skew()
    if skew_val > 1 or skew_val < -1:
        # fit_transform returns 2D array, flatten to 1D
        df_2[col] = pt.fit_transform(df_2[[col]])
df_2


,login_count_1,login_count_2,login_count_3,login_count_4,login_lat_1,login_lon_1,login_lat_2,login_lon_2,login_lat_3,login_lon_3,...,guild_changes_1,guild_changes_2,guild_changes_3,guild_changes_4,account_age_days,behavior_stability,password_changed,vpn_usage,suspicious_login_time,mass_item_sale
0,-1.172321,-0.694309,-1.272356,-0.027409,-0.507350,1.257268,-0.521818,1.252540,-0.558975,1.263964,...,-0.010478,0.878938,0.873266,-0.574365,-0.710267,-0.019890,-0.247295,2.346574,-0.224101,-0.164982
1,0.093969,-0.397719,-0.055492,-0.027409,1.438112,0.250109,1.446424,0.263228,1.437405,0.259705,...,-1.254813,-1.785353,-1.784413,-0.574365,1.241164,-1.319865,-0.247295,-0.426153,-0.224101,-0.164982
2,-1.443706,-0.856907,-1.320300,-1.144926,0.244840,0.400920,0.247543,0.401096,0.250127,0.401051,...,-1.254813,-0.009159,-1.784413,-1.629031,-0.023761,-0.412038,-0.247295,-0.426153,-0.224101,-0.164982
3,0.965498,0.627835,0.592446,1.052319,-0.853476,1.769162,-0.864894,1.765764,-0.841000,1.775059,...,1.233857,0.878938,-0.012627,1.534967,0.009849,-1.016011,-0.247295,-0.426153,-0.224101,-0.164982
4,-0.345447,0.458897,-0.142225,0.392356,-1.391197,1.659229,-1.407485,1.646771,-1.422565,1.649907,...,-0.010478,-1.785353,-0.898520,-1.629031,-1.250001,-0.166096,-0.247295,-0.426153,-0.224101,-0.164982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25884,-0.027157,0.106168,0.080793,0.458489,0.324243,1.712819,0.339177,1.706732,0.317317,1.704383,...,1.233857,1.767034,-0.012627,0.480301,1.623089,-0.972092,-0.247295,2.346574,-0.224101,-0.164982
25885,0.003515,0.861731,0.294432,0.532522,-0.612325,-0.134495,-0.617259,-0.126023,-0.604366,-0.120702,...,1.233857,1.767034,0.873266,1.534967,0.773793,0.670377,-0.247295,2.346574,-0.224101,-0.164982
25886,1.478828,0.606518,1.990725,1.128227,-1.761976,-0.552557,-1.743119,-0.560693,-1.748472,-0.554832,...,1.233857,1.767034,-0.012627,0.480301,-0.060359,-1.309482,-0.247295,-0.426153,-0.224101,-0.164982
25887,0.806884,0.301555,-0.020648,-0.027409,-0.778984,-1.309735,-0.748188,-1.295447,-0.759922,-1.307035,...,-0.010478,-0.009159,0.873266,-0.574365,0.855350,-0.905435,-0.247295,-0.426153,-0.224101,-0.164982


PCA

In [10]:
pca = PCA(n_components=0.95)
X_pca = pd.DataFrame(pca.fit_transform(df_2))
X_pca

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
0,0.788930,1.170584,-0.821993,0.869090,1.080934,1.969821,-1.944279,-0.291387,6.245020,0.506242,...,0.172419,-0.585781,-0.227569,-0.657298,0.128888,-0.144162,-0.130026,-0.661962,0.914823,-0.288385
1,2.287666,-2.509593,1.167994,4.088230,-0.147260,-0.216195,0.209878,-3.229466,-0.545572,-1.047407,...,-0.255522,0.188633,-0.244986,0.356895,-0.217982,0.597466,-0.157209,-0.071220,-0.566771,-0.137045
2,-0.814564,-0.321955,-0.392880,1.657595,-0.121031,0.132428,2.607961,-1.666148,1.121929,-0.611814,...,0.231915,-0.296210,-0.384130,0.034971,0.162476,-0.343713,-0.159533,-0.111925,-0.391314,0.323909
3,-2.652141,2.688284,-2.054432,1.694413,-1.986029,-0.628507,0.346495,0.099638,2.009300,0.170582,...,-0.394504,-0.263698,-0.564704,-0.819515,-0.265225,0.370667,0.505830,-0.591423,0.194575,0.181239
4,-1.091251,1.594839,-2.851788,2.941784,1.193700,1.493081,-1.306765,-1.100244,-0.215223,4.851639,...,-0.781634,0.448964,-0.218855,0.857648,0.301055,-0.662784,-0.301756,0.132332,0.748057,0.694645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25884,0.291721,3.703960,0.876566,1.229917,1.046386,-1.572540,-0.292299,-0.742405,1.094261,1.379023,...,0.481249,0.410063,0.526427,0.336362,-0.044255,-0.032538,0.061371,-0.010526,0.182474,-0.155359
25885,1.872684,0.876155,-1.486506,-0.439183,-2.706548,-0.929209,3.422206,0.817272,2.359581,-0.813107,...,-1.028505,-0.226823,0.410572,-0.091443,1.024916,0.589579,-0.564461,0.382664,0.839515,-0.601873
25886,-1.985409,-1.400740,-3.366296,-0.776508,1.134640,0.573209,3.285592,1.952237,-3.113175,-0.100542,...,-0.397292,0.044103,-0.026923,0.072398,-1.046103,0.486146,-1.343414,-1.018589,-0.076765,0.553976
25887,3.425070,-2.488350,-1.251798,-0.224651,-0.473456,0.117317,0.841083,2.487776,1.435791,1.750073,...,0.469709,0.651062,-0.307121,-0.063983,0.098068,0.946442,0.728064,-0.335930,-0.253766,-0.253757


Model 1: One-Class SVM

In [11]:
ocsvm_clf = OneClassSVM(
    nu=0.25,
)

ocsvm_clf.fit(X_pca)

OneClassSVM(nu=0.25)

In [12]:
joblib.dump(ocsvm_clf, 'ocsvm_clf.pkl')

['ocsvm_clf.pkl']

Predict

In [13]:
print(ocsvm_clf.decision_function(X_pca))
y_pred_ocsvm = (ocsvm_clf.predict(X_pca)==-1).astype(int)
y_pred_ocsvm

[  -5.98952416  151.75590948  270.46672908 ... -110.43218784  169.74169187
  197.3251444 ]


array([1, 0, 0, ..., 1, 0, 0])

## Save answer

In [14]:
df_ans = pd.read_csv('../Answer.csv')

df_ans.head()

,id,task1,task2,task3,task4,task5
0,ANS00001,1.0,2.0,0.000000,0,0.0
1,ANS00002,0.0,0.0,1768.238286,0,0.0
2,ANS00003,0.0,0.0,178031.889920,4,0.0
3,ANS00004,0.0,0.0,0.000000,0,0.0
4,ANS00005,1.0,0.0,0.000000,3,0.0


In [16]:
df_ans['task5'] = y_pred_ocsvm

df_ans.to_csv('../Answer.csv', index = False)

df_ans.head()

,id,task1,task2,task3,task4,task5
0,ANS00001,1.0,2.0,0.000000,0,1
1,ANS00002,0.0,0.0,1768.238286,0,0
2,ANS00003,0.0,0.0,178031.889920,4,0
3,ANS00004,0.0,0.0,0.000000,0,0
4,ANS00005,1.0,0.0,0.000000,3,0
